In [ ]:
!pip install -q chromadb
!pip install -q streamlit
!pip install -q torch
!pip install -q jedi
!pip install -q transformers
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-postprocessor-colbert-rerank
!pip install -q llama-index-vector-stores-chroma
!pip install -q llama-index-embeddings-huggingface

In [ ]:
!pwd

/content/drive/MyDrive/RAG Project


In [ ]:
%cd "/content/drive/MyDrive/RAG Project"

/content/drive/MyDrive/RAG Project


In [ ]:
!pwd

/content/drive/MyDrive/RAG Project


In [ ]:
!pip install -q bitsandbytes

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HuggingFace')

if hf_token:
  login(token=hf_token)
  print("Logged in to HuggingFace")
else:
  print("HuggingFace login token not found")

Logged in to HuggingFace


In [ ]:
!pip install -q pyngrok

In [ ]:
!streamlit run rag_app.py &>/dev/null&

from pyngrok import ngrok
public_url = ngrok.connect('8501')
print(f"Streamlit App URL: {public_url}")

Streamlit App URL: NgrokTunnel: "https://5582-34-16-193-232.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install -q ngrok

In [ ]:
from google.colab import userdata

ngrok_auth_token = userdata.get('ngrok')

if ngrok_auth_token:
  get_ipython().system(f'ngrok config add-authtoken {ngrok_auth_token}')
  print("ngrok authtoken added")
else:
  print("ngrok authtoken not found")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok authtoken added
